In [ ]:
# inventory_loader.py
import pandas as pd
import os

def load_inventory(file_path: str):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Inventory file not found: {file_path}")

    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path)
    elif file_path.endswith(".xlsx") or file_path.endswith(".xls"):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Use CSV or XLSX.")

    # Expecting columns: sku, name, stock, threshold
    df.columns = [col.lower() for col in df.columns]

    inventory_dict = {}

    for _, row in df.iterrows():
        inventory_dict[row["sku"]] = {
            "name": row["name"],
            "stock": int(row["stock"]),
            "threshold": int(row["threshold"]),
        }

    return inventory_dict


In [ ]:
# graph_maas_llm.py
import os
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from inventory_loader import load_inventory

load_dotenv()

class InventoryState(dict):
    pass

# Load inventory from file (CSV or XLSX)
INVENTORY_FILE = "inventory.xlsx"   # <-- change to "inventory.csv" if needed
inventory = load_inventory(INVENTORY_FILE)

# Azure MaaS LLM
llm = ChatOpenAI(
    model="azure/genailab-maas-gpt-4o-mini",
    api_key=os.getenv("AZURE_MAAS_API_KEY"),
    base_url="https://models.inference.ai.azure.com",
    temperature=0
)

# 1️⃣ Node: Fetch inventory data for SKU
def fetch_inventory(state: InventoryState):
    sku = state["sku"]

    if sku not in inventory:
        raise ValueError(f"SKU '{sku}' not found in inventory file.")

    state["item_name"] = inventory[sku]["name"]
    state["stock"] = inventory[sku]["stock"]
    state["threshold"] = inventory[sku]["threshold"]

    return state

# 2️⃣ Node: LLM decides reorder + quantity
def llm_replenishment_planner(state: InventoryState):
    sku = state["sku"]
    name = state["item_name"]
    stock = state["stock"]
    threshold = state["threshold"]

    prompt = f"""
    You are an inventory planning agent.

    SKU: {sku}
    Item: {name}
    Current Stock: {stock}
    Reorder Threshold: {threshold}

    Based on stock levels:
    - Should we reorder this item?
    - If yes, suggest an optimal reorder quantity.
    - Provide a short explanation.

    Answer in concise bullet points.
    """

    resp = llm.invoke(prompt)
    state["llm_response"] = resp.content
    return state


# ---------- Build LangGraph workflow ----------
workflow = StateGraph(InventoryState)

workflow.set_entry_point("fetch_inventory")
workflow.add_node("fetch_inventory", fetch_inventory)
workflow.add_node("llm_planner", llm_replenishment_planner)

workflow.add_edge("fetch_inventory", "llm_planner")
workflow.add_edge("llm_planner", END)

app = workflow.compile()

